# Bayesian LSTM

In [1]:
import gc
from pathlib import Path
import numpy as np
import pickle

import torch
from torch.utils.data import Dataset, DataLoader
import mlflow

import blitz.modules as blitz_modules
from blitz.utils import variational_estimator

from sklearn.metrics import mean_absolute_error, root_mean_squared_error

import copy
import optuna

import json
from mlflow.models.signature import ModelSignature
from mlflow.types import Schema, TensorSpec

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


DATA_PATH = Path("../../data/processed")

EXPERIMENT_NAME = "fs_06_load_calendar_future_weather"
OUTPUT_PATH = DATA_PATH / "ml_data" / EXPERIMENT_NAME
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

In [3]:
# Possible fix for stability in searches
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

print("cudnn.enabled       =", torch.backends.cudnn.enabled)
print("cudnn.benchmark     =", torch.backends.cudnn.benchmark)
print("cudnn.deterministic =", torch.backends.cudnn.deterministic)

cudnn.enabled       = True
cudnn.benchmark     = False
cudnn.deterministic = True


## First: Start the MLFlow-Server for logging the experiments

Run the following command: 

Powershell:

mlflow server ` 
    --backend-store-uri sqlite:///mlflow.db ` 
    --default-artifact-root ./mlartifacts/  


Bash / Git Bash / WSL / Linux / macOS
mlflow server \
  --backend-store-uri sqlite:///mlflow.db \
  --default-artifact-root ./mlartifacts


In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("load_forecasting_bayesian_lstm")

mlflow.enable_system_metrics_logging()

## Defining the model class

In [5]:
@variational_estimator
class BayesianLSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, horizon):
        super().__init__()
        self.lstm = blitz_modules.BayesianLSTM(
            in_features=input_size,
            out_features=hidden_size
        )

        self.fc = torch.nn.Linear(hidden_size, 2 * horizon) # times two, because we need to output a mean and variance for the GausianNLLLoss
    
    def forward(self, x):
        # OLD (incorrect for BLiTZ BayesianLSTM):
        # x is in format (batch_size, seq_len, input_size) but (seq_len, batch_size, input_size) is required
        # x = x.transpose(0, 1) # now in the correct format
        # out, _ = self.lstm(x) # (seq_len, batch_size, input_size)
        # h = out[-1, :, :] # The last hidden state (batch_size, hidden_size)
        # return self.fc(h) # (batch, 2 * horizon) each batch having the shape [mu_1, mu_2, ... var_1, var_2 ...]

        # Correct: BLiTZ BayesianLSTM expects (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)
        h = out[:, -1, :]  # last time step for each sample
        return self.fc(h)  # (batch, 2 * horizon)

## Load the data

In [6]:
# Load the scaler
scalers_dir = OUTPUT_PATH / "scalers"

with open(scalers_dir / "y_scaler.pkl", "rb") as f:
    y_scaler = pickle.load(f)

## Define the dataset

In [7]:
class TimeSeriesDataset(Dataset):
    """PyTorch Dataset for time series forecasting."""
    
    def __init__(self, X, y):
        """
        Args:
            X: Input sequences (n_samples, window_size, n_features)
            y: Target values (n_samples, forecast_horizon)
        """
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).float()
    
    def __len__(self):
        
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


X_train_seq = np.load(OUTPUT_PATH / "X_train.npy")
y_train_seq = np.load(OUTPUT_PATH / "y_train.npy")
train_dataset = TimeSeriesDataset(X_train_seq, y_train_seq)
# del X_train_seq, y_train_seq
gc.collect()

X_val_seq = np.load(OUTPUT_PATH / "X_val.npy")
y_val_seq = np.load(OUTPUT_PATH / "y_val.npy")
val_dataset = TimeSeriesDataset(X_val_seq, y_val_seq)
del X_val_seq, y_val_seq
gc.collect()

X_test_seq = np.load(OUTPUT_PATH / "X_test.npy")
y_test_seq = np.load(OUTPUT_PATH / "y_test.npy")
test_dataset = TimeSeriesDataset(X_test_seq, y_test_seq)
del X_test_seq, y_test_seq
gc.collect()

print(f"Train dataset: {len(train_dataset)} samples")
print(f"Val dataset:   {len(val_dataset)} samples")
print(f"Test dataset:  {len(test_dataset)} samples")
print(f"Sample shape: X={train_dataset[0][0].shape}, y={train_dataset[0][1].shape}")

                

Train dataset: 164214 samples
Val dataset:   39863 samples
Test dataset:  39863 samples
Sample shape: X=torch.Size([672, 7]), y=torch.Size([96])


## Hyperparameter tuning

In [ ]:
def objective(trial):
    with mlflow.start_run(run_name=f"trial_{trial.number}", nested=True):
        hidden_size = trial.suggest_int("hidden_size", 64, 256, step=32)
        lr = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
        batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
        weight_decay = trial.suggest_float("weight_decay", 1e-7, 1e-3, log=True)
        sample_nbr = trial.suggest_categorical("sample_nbr", [3, 5, 7, 10])
        eval_sample_nbr = 50
        kl_warmup_epochs = trial.suggest_int("kl_warmup_epochs", 5, 25)

        mlflow.log_params(trial.params)
        mlflow.log_param("eval_sample_nbr", eval_sample_nbr)

        model = BayesianLSTM(
            input_size=train_dataset[0][0].shape[-1],
            hidden_size=hidden_size,
            horizon=train_dataset[0][1].shape[-1],
        ).to(DEVICE)

        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=lr,
            weight_decay=weight_decay,
        )

        gauss_nll = torch.nn.GaussianNLLLoss(eps=1e-6, reduction="mean") #torch.nn.MSELoss()

        def nll_criterion(pred_params, y):
            mu, log_var = torch.chunk(pred_params, 2, dim=-1)
            var = torch.nn.functional.softplus(log_var) + 1e-6
            return gauss_nll(mu, y, var)

        train_dataloader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True,
        )

        val_dataloader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=False,
        )

        base_kl_weight = 1.0 / len(train_dataset)
        best_val = float("inf")
        best_state = None
        best_epoch = -1
        patience = 10
        wait = 0
        num_epochs = 50

        for epoch in range(num_epochs):
            model.train()
            train_elbo = 0.0

            kl_weight = base_kl_weight * min(1.0, (epoch + 1) / kl_warmup_epochs)

            for xb, yb in train_dataloader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)

                optimizer.zero_grad()

                elbo = model.sample_elbo(
                    inputs=xb,
                    labels=yb,
                    criterion=nll_criterion,
                    sample_nbr=sample_nbr,
                    complexity_cost_weight=kl_weight,
                )

                elbo.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                train_elbo += elbo.item()

            train_elbo /= len(train_dataloader)
            mlflow.log_metric("train_elbo", train_elbo, step=epoch)

            model.eval()
            val_elbo = 0.0
            val_nll = 0.0
            val_mu_preds = []
            val_targets = []

            with torch.no_grad():
                for xb, yb in val_dataloader:
                    xb, yb = xb.to(DEVICE), yb.to(DEVICE)

                    elbo = model.sample_elbo(
                        inputs=xb,
                        labels=yb,
                        criterion=nll_criterion,
                        sample_nbr=sample_nbr,
                        complexity_cost_weight=kl_weight,
                    )

                    pred_samples = torch.stack([model(xb) for _ in range(eval_sample_nbr)], dim=0)
                    mu_s, log_var_s = torch.chunk(pred_samples, 2, dim=-1)
                    var_s = torch.nn.functional.softplus(log_var_s) + 1e-6
                    mu_pred = mu_s.mean(dim=0)
                    var_pred = var_s.mean(dim=0) + mu_s.var(dim=0, unbiased=False)
                    nll = gauss_nll(mu_pred, yb, var_pred)

                    val_mu_preds.append(mu_pred.detach().cpu().numpy())
                    val_targets.append(yb.detach().cpu().numpy())

                    val_elbo += elbo.item()
                    val_nll += nll.item()

            val_elbo /= len(val_dataloader)
            val_nll /= len(val_dataloader)
            y_pred_val = np.concatenate(val_mu_preds, axis=0)
            y_true_val = np.concatenate(val_targets, axis=0)
            val_mae = mean_absolute_error(y_true_val, y_pred_val)
            val_rmse = root_mean_squared_error(y_true_val, y_pred_val)
            val_mse = val_rmse ** 2
            val_rmse_h = [
                root_mean_squared_error(y_true_val[:, h], y_pred_val[:, h])
                for h in [0, 3, 11, 23, 47, 95]
            ]

            mlflow.log_metric("val_elbo", val_elbo, step=epoch)
            mlflow.log_metric("val_nll", val_nll, step=epoch)
            mlflow.log_metric("val_mse", val_mse, step=epoch)
            mlflow.log_metric("val_mae", val_mae, step=epoch)
            mlflow.log_metric("val_rmse", val_rmse, step=epoch)
            for h_idx, rmse_h in enumerate(val_rmse_h):
                mlflow.log_metric(f"val_rmse_h{h_idx+1}", rmse_h, step=epoch)
            mlflow.log_metric("kl_weight", kl_weight, step=epoch)

            trial.report(val_nll, epoch)

            if trial.should_prune():
                del model, optimizer
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                raise optuna.TrialPruned()

            if val_nll < best_val:
                best_val = val_nll
                wait = 0
                best_state = copy.deepcopy(model.state_dict())
                best_epoch = epoch
            else:
                wait += 1

            if wait >= patience:
                break

            print(
                
                f"Epoch {epoch+1}/{num_epochs}, "
                f"Train ELBO: {train_elbo:.4f}, "
                f"Val ELBO: {val_elbo:.4f}, "
                f"Val NLL: {val_nll:.6f}, "
                f"Val MSE: {val_mse:.6f}, "
                f"Val MAE: {val_mae:.6f}, "
                f"KL w: {kl_weight:.2e}"
            )

        if best_state is not None:
            model.load_state_dict(best_state)
            mlflow.log_metric("best_epoch", best_epoch)
            mlflow.pytorch.log_model(model, artifact_path="best_model")

        mlflow.log_metric("best_val_nll", best_val)

        del model, optimizer
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        return best_val



In [ ]:
STUDY_NAME = f"bayesian_lstm_nll_{EXPERIMENT_NAME}"
STUDY_STORAGE = "sqlite:///optuna_bayesian_lstm.db"

study = optuna.create_study(
    study_name=STUDY_NAME,
    storage=STUDY_STORAGE,
    load_if_exists=True,
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=1234),
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
)

with mlflow.start_run(run_name=f"optuna_search_bayesian_lstm_nll_{EXPERIMENT_NAME}"):
    study.optimize(objective, n_trials=50, show_progress_bar=True, gc_after_trial=True)
    mlflow.log_metric("best_val_nll", study.best_value)
    mlflow.log_params(study.best_params)
    mlflow.log_param("best_trial_number", study.best_trial.number)

best_params = study.best_params
best_params



## Final training after the hyperparameter tuning

In [8]:
meta_path = OUTPUT_PATH / "meta.json"
with open(meta_path, "r", encoding="utf-8") as f:
    meta = json.load(f)

seq_len = meta["window_size"]          # 672
n_features = meta["n_features"]        # num_feautes
horizon = meta["forecast_horizon"]     # 96

assert X_train_seq.shape[1:] == (seq_len, n_features)
assert y_train_seq.shape[1] == horizon

input_example = X_train_seq[:2].astype(np.float32)

signature = ModelSignature(
    inputs=Schema([
        TensorSpec(np.dtype(np.float32), (-1, seq_len, n_features), name="x")
    ]),
    outputs=Schema([
        TensorSpec(np.dtype(np.float32), (-1, horizon), name="yhat")
    ]),
)

In [9]:
BATCH_SIZE = 64
NUM_EPOCHS = 60
HIDDEN_SIZE = 64
SAMPLE_NBR = 3
VAL_EVAL_SAMPLE_NBR = 10
TEST_SAMPLE_NBR = 70
KL_WARMUP_EPOCHS = 30

optim_kwargs = {
    "lr": 3e-4,
    "weight_decay": 1e-5,
}

model = BayesianLSTM(
    input_size=train_dataset[0][0].shape[-1],
    horizon=train_dataset[0][1].shape[-1],
    hidden_size=HIDDEN_SIZE
).to(DEVICE)

optim = torch.optim.Adam(
    params=model.parameters(),
    **optim_kwargs
)

gauss_nll = torch.nn.GaussianNLLLoss(eps=1e-6, reduction="mean") #torch.nn.MSELoss()

def nll_criterion(pred_params, y):
    mu, log_var = torch.chunk(pred_params, 2, dim=-1)
    var = torch.nn.functional.softplus(log_var) + 1e-3
    return gauss_nll(mu, y, var)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=False
)

# Scale KL term by dataset size; warm up to full weight over KL_WARMUP_EPOCHS.
base_kl_weight =  1.0 / len(train_dataset)

best_val = float("inf")
patience = 5
wait = 0
with mlflow.start_run(run_name=f"baysian_lstm_nll_fs_06_load_calendar_future_weather"):
    mlflow.log_params(optim_kwargs)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("hidden_size", HIDDEN_SIZE)
    mlflow.log_param("sample_nbr", SAMPLE_NBR)
    mlflow.log_param("val_eval_sample_nbr", VAL_EVAL_SAMPLE_NBR)
    mlflow.log_param("kl_warmup_epochs", KL_WARMUP_EPOCHS)
    mlflow.log_param("base_kl_weight", base_kl_weight)

    for epoch in range(NUM_EPOCHS):

        model.train()
        train_elbo = 0.0

        kl_weight = base_kl_weight * min(1.0, (epoch + 1) / KL_WARMUP_EPOCHS)

        for xb, yb in train_dataloader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)

            optim.zero_grad()

            elbo = model.sample_elbo(
                inputs=xb,
                labels=yb,
                criterion=nll_criterion,
                sample_nbr=SAMPLE_NBR,
                complexity_cost_weight=kl_weight
            )

            elbo.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optim.step()

            train_elbo += elbo.item()

        train_elbo /= len(train_dataloader)

        mlflow.log_metric("train_elbo", train_elbo, step=epoch)

        # validate only 3 epochs
        if epoch % 3 == 0:
            model.eval()
            val_elbo = 0.0
            val_nll = 0.0

            val_var_mean_acc = 0.0
            val_var_min_acc = float("inf")
            val_var_max_acc = -float("inf")

            with torch.no_grad():
                for xb, yb in val_dataloader:
                    xb, yb = xb.to(DEVICE), yb.to(DEVICE)

                    elbo = model.sample_elbo(
                        inputs=xb,
                        labels=yb,
                        criterion=nll_criterion,
                        sample_nbr=SAMPLE_NBR,
                        complexity_cost_weight=kl_weight
                    )

                    pred_samples = torch.stack([model(xb) for _ in range(VAL_EVAL_SAMPLE_NBR)], dim=0)
                    mu_s, log_var_s = torch.chunk(pred_samples, 2, dim=-1)
                    var_s = torch.nn.functional.softplus(log_var_s) + 1e-6
                    mu_pred = mu_s.mean(dim=0)
                    var_pred = var_s.mean(dim=0) + mu_s.var(dim=0, unbiased=False)
                    nll = gauss_nll(mu_pred, yb, var_pred)

                    val_elbo += elbo.item()
                    val_nll += nll.item()

                    val_var_mean_acc += var_pred.mean().item()
                    val_var_min_acc = min(val_var_min_acc, var_pred.min().item())
                    val_var_max_acc = max(val_var_max_acc, var_pred.max().item())

                val_elbo /= len(val_dataloader)
                val_nll /= len(val_dataloader)
                val_var_mean = val_var_mean_acc / len(val_dataloader)

                mlflow.log_metric("val_elbo", val_elbo, step=epoch)
                mlflow.log_metric("val_nll", val_nll, step=epoch)
                mlflow.log_metric("kl_weight", kl_weight, step=epoch)
                
                mlflow.log_metric("val_var_mean", val_var_mean, step=epoch)
                mlflow.log_metric("val_var_min", val_var_min_acc, step=epoch)
                mlflow.log_metric("val_var_max", val_var_max_acc, step=epoch)

                if val_nll < best_val:
                    best_val = val_nll
                    wait = 0
                    best_state = copy.deepcopy(model.state_dict())
                    best_epoch = epoch

                else:
                    wait +=1
                
                if wait >= patience:
                    break

                print(
                    f"Epoch {epoch+1}/{NUM_EPOCHS}, "
                    f"Train ELBO: {train_elbo:.4f}, "
                    f"Val ELBO: {val_elbo:.4f}, "
                    f"Val NLL: {val_nll:.6f}, "
                    f"KL w: {kl_weight:.2e}"
                )

    if best_state is not None:
        model.load_state_dict(best_state)
        model.eval()
        mlflow.log_metric("best_epoch", best_epoch)

        mlflow.log_dict(meta, "artifacts/data_meta.json")
        mlflow.log_artifact(str(scalers_dir / "y_scaler.pkl"), artifact_path="preprocessing")

        mlflow.pytorch.log_model(
            pytorch_model=model,
            name="best_model",
            signature=signature,
            input_example=input_example,
            metadata={
                "feature_names": meta["feature_names"],
                "target": "actual_load_mw",
                "scaler_type": meta["scaler_type"],
                "output_space": "scaled_target",
            },
            tags={"feature_set": EXPERIMENT_NAME},
        )

        best_model = model

        best_model.eval()

        all_preds = []
        all_trues = []

        with torch.no_grad():
            for xb, yb in test_dataloader:
                xb = xb.to(DEVICE)
                yb = yb.to(DEVICE)

                samples = torch.stack([best_model(xb) for _ in range(TEST_SAMPLE_NBR)], dim=0)  # [S, N, 2H]
                mu_s, log_var_s = torch.chunk(samples, 2, dim=-1)                                # [S, N, H], [S, N, H]
                var_s = torch.nn.functional.softplus(log_var_s) + 1e-6

                mu_scaled = mu_s.mean(dim=0)  # point forecast in scaled space, [N, H]
                pred_var_scaled = var_s.mean(dim=0) + mu_s.var(dim=0, unbiased=False)  # total predictive var

                N, H = mu_scaled.shape

                y_pred = y_scaler.inverse_transform(mu_scaled.cpu().numpy().reshape(-1, 1)).reshape(N, H)
                y_true = y_scaler.inverse_transform(yb.cpu().numpy().reshape(-1, 1)).reshape(N, H)

                all_preds.append(y_pred)
                all_trues.append(y_true)

        # concat across batches
        y_pred_full = np.concatenate(all_preds, axis=0)
        y_true_full = np.concatenate(all_trues, axis=0)

        mae = mean_absolute_error(y_true_full, y_pred_full)
        rmse = root_mean_squared_error(y_true_full, y_pred_full)

        mlflow.log_metric("test_mae_unscaled", mae)
        mlflow.log_metric("test_rmse_unscaled", rmse)

        for h in [0, 3, 11, 23, 47, 95]:
            mlflow.log_metric(
                f"rmse_h{h+1}",
                root_mean_squared_error(y_true_full[:, h], y_pred_full[:, h])
            )

2026/02/11 15:29:31 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2026/02/11 15:29:31 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Epoch 1/60, Train ELBO: -0.8950, Val ELBO: -1.0356, Val NLL: -1.078038, KL w: 2.03e-07
Epoch 4/60, Train ELBO: -1.2710, Val ELBO: -1.1639, Val NLL: -1.273162, KL w: 8.12e-07
Epoch 7/60, Train ELBO: -1.3009, Val ELBO: -1.1345, Val NLL: -1.293617, KL w: 1.42e-06
Epoch 10/60, Train ELBO: -1.3518, Val ELBO: -1.1672, Val NLL: -1.369472, KL w: 2.03e-06
Epoch 13/60, Train ELBO: -1.3183, Val ELBO: -1.1042, Val NLL: -1.379954, KL w: 2.64e-06
Epoch 16/60, Train ELBO: -1.2767, Val ELBO: -1.1076, Val NLL: -1.423049, KL w: 3.25e-06
Epoch 19/60, Train ELBO: -1.2287, Val ELBO: -1.0540, Val NLL: -1.439635, KL w: 3.86e-06
Epoch 22/60, Train ELBO: -1.1782, Val ELBO: -0.9934, Val NLL: -1.450022, KL w: 4.47e-06
Epoch 25/60, Train ELBO: -1.1302, Val ELBO: -0.9691, Val NLL: -1.475837, KL w: 5.07e-06
Epoch 28/60, Train ELBO: -1.0850, Val ELBO: -0.9302, Val NLL: -1.479073, KL w: 5.68e-06
Epoch 31/60, Train ELBO: -1.0577, Val ELBO: -0.8847, Val NLL: -1.475613, KL w: 6.09e-06
Epoch 34/60, Train ELBO: -1.0612, V

2026/02/14 09:27:10 WARNING mlflow.utils.requirements_utils: Found torch version (2.9.1+cu128) contains a local version label (+cu128). MLflow logged a pip requirement for this package as 'torch==2.9.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/14 09:27:25 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


2026/02/14 09:27:25 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": [
    [
      [
        -0.879765391.... Alternatively, you can avoid passing input example and pass model signature instead when logging the model. To ensure the input example is valid prior to serving, please try calling `mlflow.models.validate_serving_input` on the model uri and serving input example. A serving input example can be generated from model input example using `mlflow.models.convert_input_example_to_serving_input` function.
Got error: The PyTorch flavor does not support List or Dict input types. Please use a pandas.DataFrame or a numpy.ndarray
2026/02/14 09:27:25 INFO mlflow.models.model: Found the following environment variables used during model inference: [CDSAPI_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
2026/02/14 11:23:29 INFO mlflow.s

🏃 View run baysian_lstm_nll_fs_06_load_calendar_future_weather at: http://127.0.0.1:5000/#/experiments/3/runs/2ccd3a12be4d4ec484be5af5fc8ab4e3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


## Evaluation on the test set

In [ ]:
ranked_checkpoints = mlflow.search_logged_models(
    filter_string=f"source_run_id='0df404bd6fee454788a60bacb548f643'",
    order_by=[{"field_name": "metrics.val_nll", "ascending": True}],
    output_format="list",
)

best_checkpoint = ranked_checkpoints[0]
best_checkpoint

LoggedModel(artifact_location='mlflow-artifacts:/3/models/m-59d7d88d721247bc91c1ced7635a401f/artifacts', creation_timestamp=1770719453301, experiment_id='3', last_updated_timestamp=1770719462357, model_id='m-59d7d88d721247bc91c1ced7635a401f', model_type='', model_uri='models:/m-59d7d88d721247bc91c1ced7635a401f', name='best_model', source_run_id='0df404bd6fee454788a60bacb548f643', status=<LoggedModelStatus.READY: 'READY'>, status_message='')

In [19]:
loaded_model = mlflow.pytorch.load_model(best_checkpoint.model_uri)

In [ ]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=True
)
loaded_model.eval()

all_preds = []
all_trues = []

with torch.no_grad():
    for xb, yb in test_dataloader:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)

        samples = torch.stack([loaded_model(xb) for _ in range(TEST_SAMPLE_NBR)], dim=0)  # [S, N, 2H]
        mu_s, log_var_s = torch.chunk(samples, 2, dim=-1)                                # [S, N, H], [S, N, H]
        var_s = torch.nn.functional.softplus(log_var_s) + 1e-6

        mu_scaled = mu_s.mean(dim=0)  # point forecast in scaled space, [N, H]
        pred_var_scaled = var_s.mean(dim=0) + mu_s.var(dim=0, unbiased=False)  # total predictive var

        N, H = mu_scaled.shape

        y_pred = y_scaler.inverse_transform(mu_scaled.cpu().numpy().reshape(-1, 1)).reshape(N, H)
        y_true = y_scaler.inverse_transform(yb.cpu().numpy().reshape(-1, 1)).reshape(N, H)

        all_preds.append(y_pred)
        all_trues.append(y_true)

# concat across batches
y_pred_full = np.concatenate(all_preds, axis=0)
y_true_full = np.concatenate(all_trues, axis=0)

mae = mean_absolute_error(y_true_full, y_pred_full)
rmse = root_mean_squared_error(y_true_full, y_pred_full)

mlflow.log_metric("test_mae_unscaled", mae)
mlflow.log_metric("test_rmse_unscaled", rmse)

for h in [0, 3, 11, 23, 47, 95]:
    mlflow.log_metric(
        f"rmse_h{h+1}",
        root_mean_squared_error(y_true_full[:, h], y_pred_full[:, h])
    )



2026/02/05 12:26:31 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2026/02/05 12:26:31 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
